## tradeSeq for DE along Mullerian rostro-caudal axis in female reproductive tract

In [ ]:
#if (!require("BiocManager", quietly = TRUE))
#    install.packages("BiocManager")

#BiocManager::install("tradeSeq")

In [ ]:
#install.packages('igraph')

In [ ]:
ip = as.data.frame(installed.packages()[,c(1,3:4)])
ip = ip[is.na(ip$Priority),1:2,drop=FALSE]
head(ip)

In [ ]:
library(tradeSeq)
library(SingleCellExperiment)
library(slingshot)

In [ ]:
library(SoupX)

In [ ]:
sessionInfo()

In [ ]:
adata <- readRDS("/lustre/scratch126/cellgen/team292/vl6/VISIUM/femalereproductiveaxis_mese_cytassist_downsampled.rds")
adata


In [ ]:
counts <- assay(adata, "X")

In [ ]:
set.seed(7)
pseudotime <- colData(adata)$FemaleReproductiveAxis
cellWeights <- rep(1, 501)

In [ ]:
#icMat <- evaluateK(counts = counts, sds = SlingshotDataSet(adata), k = 3:10, 
#                   nGenes = 200, verbose = T)

In [ ]:
#BiocManager::install("scran")

In [ ]:
library(scran)

In [ ]:
libsizes <- colSums(counts)
size.factors <- libsizes/mean(libsizes)
logcounts(adata) <- log2(t(t(counts)/size.factors) + 1)
assayNames(adata)

In [ ]:
# Filter genes with non-zero biological components
getVariableGenes <- function(adata,fdr_thresh=NULL,var_thresh=0) {
  gene_var <- modelGeneVar(adata)
  genes <- getTopHVGs(gene_var, var.threshold =var_thresh ,fdr.threshold = fdr_thresh)
  return(genes)
}

In [ ]:
genes <- getVariableGenes(adata)

In [ ]:
length(genes)

In [ ]:
dim(counts[genes,])

In [ ]:
sce <- fitGAM(counts = counts[genes,], pseudotime = pseudotime, cellWeights = cellWeights, nknots = 10,
                verbose = TRUE, parallel = TRUE)

In [ ]:
table(rowData(sce)$tradeSeq$converged)

In [ ]:
adata <- adata[genes, ]

In [ ]:
adata

In [ ]:
gamList <- fitGAM(counts = counts[genes,], pseudotime = pseudotime, cellWeights = cellWeights, nknots = 10,
                verbose = TRUE, 
                  parallel = TRUE, sce = FALSE)

In [ ]:
summary(gamList[["SRD5A2"]])


In [ ]:
pvalLineage <- getSmootherPvalues(gamList)
statLineage <- getSmootherTestStats(gamList)

### ASSOCIATION TEST

A first exploration of the data analysis may consist of checking whether gene expression is associated with a particular lineage. The statistical test performed here, implemented in the associationTest function, is testing the null hypothesis that all smoother coefficients are equal to each other. This can be interpreted as testing whether the average gene expression is significantly changing along pseudotime.



In [ ]:
assoRes <- associationTest(sce)
head(assoRes)

In [ ]:
write.csv(assoRes, file = '/nfs/team292/vl6/FetalReproductiveTract/manuscript_v1/supp_tables/SupplementaryTable8_müllerian_mese_cytassist.csv')


### FILTER GENES AND CLUSTER THEM BY PATTERN

In [ ]:
dim(assoRes)

In [ ]:
assoRes = assoRes[assoRes$pvalue < 0.001,]
dim(assoRes)

In [ ]:
assoRes = assoRes[assoRes$waldStat > 500,]
dim(assoRes)

In [ ]:
assoRes['HOXA13',]

In [ ]:
assoRes = assoRes[order(assoRes$waldStat, decreasing = TRUE),]

In [ ]:
tail(assoRes)

In [ ]:
# Get the index of the DataFrame
tops <- row.names(assoRes)[1:50]

In [ ]:
tops

In [ ]:
library(ggplot2)

In [ ]:
for (gene in tops) {
    print(gene)
    print(plotSmoothers(sce, counts, gene = gene, alpha = 0) + ggtitle(gene) +
                          theme(title = element_text(size = 20, face = 'bold'),
                                axis.title = element_text(size = 20),
                                axis.text = element_text(size = 20),
                                legend.text = element_text(size = 20),
                                legend.title = element_text(size = 20, face = "bold")))
}

In [ ]:
sce

In [ ]:
model_vals_df <- predictSmooth(models = sce, gene=row.names(assoRes), nPoints=100) # output is 715 genes x 100 pseudospace values 

In [ ]:
model_vals_mtx <- predictSmooth(models = sce, gene=row.names(assoRes), nPoints=100, tidy=FALSE)

In [ ]:
dim(model_vals_mtx) # 715 genes x 100 pseudospace values 

In [ ]:
model_vals_mtx[1:10, 1:20]

In [ ]:
write.csv(model_vals_mtx, file = '/lustre/scratch126/cellgen/team292/vl6/VISIUM/femalereproductiveaxis_mese_cytassist_downsampled_fitted_values_tradeseq.csv')


In [ ]:
hoxa5 <- plotSmoothers(sce, counts, gene = 'HOXA5', alpha = 0, 
                       xlab = "Female Reproductive Axis",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("HOXA5") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()
hoxa5

In [ ]:
hoxa9 <- plotSmoothers(sce, counts, gene = 'HOXA9', alpha = 0, 
                       xlab = "Female Reproductive Axis",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("HOXA9") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()
hoxa9

In [ ]:
hoxa10 <- plotSmoothers(sce, counts, gene = 'HOXA10', alpha = 0, 
                       xlab = "Female Reproductive Axis",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("HOXA5") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()
hoxa10

In [ ]:
hoxc6 <- plotSmoothers(sce, counts, gene = 'HOXC6', alpha = 0, 
                       xlab = "Female Reproductive Axis",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("HOXA5") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()
hoxc6

In [ ]:
hoxa11 <- plotSmoothers(sce, counts, gene = 'HOXA11', alpha = 0, 
                       xlab = "Female Reproductive Axis",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("HOXA5") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()
hoxa11

In [ ]:
hoxa7 <- plotSmoothers(sce, counts, gene = 'HOXA7', alpha = 0, 
                       xlab = "Female Reproductive Axis",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("HOXA7") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()
hoxa7

In [ ]:
hoxa13 <- plotSmoothers(sce, counts, gene = 'HOXA13', alpha = 0, 
                       xlab = "Female Reproductive Axis",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("HOXA13") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()
hoxa13

In [ ]:
p <- grid.arrange(hoxa5, hoxa9, hoxc5, hoxa10, HOXC6,  hoxa11, hoxa7,     hoxa13, 
                  nrow = 4, ncol = 2)

In [ ]:
ggsave('/home/jovyan/RepTract/visium_femalereproductiveaxis_hox_mese_tradeseq.pdf',
       plot = p,
       width = 8, height = 13, dpi = 300)

In [ ]:
BMP4 <- plotSmoothers(sce, counts, gene = 'BMP4', alpha = 0, 
                       xlab = "Female Reproductive Axis",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("BMP4") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()
BMP4

In [ ]:
IGF1 <- plotSmoothers(sce, counts, gene = 'IGF1', alpha = 0, 
                       xlab = "Female Reproductive Axis",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("IGF1") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()
IGF1

In [ ]:
SOX17 <- plotSmoothers(sce, counts, gene = 'SOX17', alpha = 0, 
                       xlab = "Female Reproductive Axis",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("SOX17") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light() + ylim(0, 3.5)
SOX17

In [ ]:
CSF1R <- plotSmoothers(sce, counts, gene = 'CSF1R', alpha = 0, 
                       xlab = "Female Reproductive Axis",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("CSF1R") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light() + ylim(0, 3.2)
CSF1R

In [ ]:
PNOC <- plotSmoothers(sce, counts, gene = 'PNOC', alpha = 0, 
                       xlab = "Female Reproductive Axis",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("PNOC") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light() + ylim(0, 3.7)
PNOC

In [ ]:
p2 <- grid.arrange(PNOC, SOX17, CSF1R,
                  nrow = 3, ncol = 1)

In [ ]:
ggsave('/home/jovyan/RepTract/visium_femalereproductiveaxis_spurious_mese_tradeseq.pdf',
       plot = p2,
       width = 4, height = 9, dpi = 300)

In [ ]:
TNC <- plotSmoothers(sce, counts, gene = 'TNC', alpha = 0, 
                       xlab = "Female Reproductive Axis",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("TNC") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()
TNC

In [ ]:
SRD5A2 <- plotSmoothers(sce, counts, gene = 'SRD5A2', alpha = 0, 
                       xlab = "Female Reproductive Axis",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("SRD5A2") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()
SRD5A2


In [ ]:
GDF10 <- plotSmoothers(sce, counts, gene = 'GDF10', alpha = 0, 
                       xlab = "Female Reproductive Axis",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("GDF10") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()
GDF10

In [ ]:
PTGES <- plotSmoothers(sce, counts, gene = 'PTGES', alpha = 0, 
                       xlab = "Female Reproductive Axis",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("PTGES") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()
PTGES